In [9]:
import random
from collections import namedtuple

Nimply = namedtuple("Nimply", "row, num_objects")
Gene = namedtuple("Gene", ["condition","action","place"])
Individual = namedtuple("Individual", ["genome", "fitness"])

## Global Variables

In [10]:
eval_amount = 1000
genome_size = 3
population_size = 100
offspring_size = 50
generations = 1000
mutation_rate = 0.5

In [11]:
def nimSum(rows):
    sum=rows[0]
    for i in rows[1:]:
        sum^=i
    return sum

def checkMisere(rows):
    cnt=0
    nonOneRow=-1
    for i,row in enumerate(rows):
        if row>=2:
            cnt+=1
            nonOneRow=i
        if cnt==2:
            return -1
    return nonOneRow

## Nim

In [12]:
class Nim:
    def __init__(self, num_rows: int, k: int = None) -> None:
        self._rows = [2*i + 1 for i in range(num_rows)]
        self._k = k

    def nimming(self, ply:Nimply) -> None:
        if ply is None:
            for i,_ in self._rows:
                self._rows[i] = 0
            return
        row, num_objects=ply
        assert num_objects>0
        assert self._rows[row] >= num_objects
        assert self._k is None or num_objects <= self._k
        self._rows[row] -= num_objects

    def __bool__(self):
        return sum(self._rows) > 0
    def __str__(self):
        return "<" + " ".join(str(_) for _ in self._rows) + ">"
    @property
    def rows(self) -> tuple:
        return tuple(self._rows)
    @property
    def k(self) -> int:
        return self._k

    def endTest(self):
        if sum(self._rows) == 0:
            print("You lost")
            return 1
        return 0

    def board(self)->None:
        for i,row in enumerate(self._rows):
            print(i,":",end=" ")
            for j in range(row):
                print("|", end=' ')
            print("\n")

## Condition

In [13]:
def even_elems_if(state: Nim) -> bool:
    return sum(state.rows)%2 == 0

def odd_elems_if(state: Nim) -> bool:
    return sum(state.rows)%2 != 0

def even_stacks_if(state: Nim) -> bool:
    return len(state.rows)%2 == 0

def odd_stacks_if(state: Nim) -> bool:
    return len(state.rows)%2 != 0

def nimsum_if(state: Nim) -> bool:
    return nimSum(state.rows)!=0 

conditions = {"even_elems": even_elems_if, "odd_elems": odd_elems_if, "even_stacks": even_stacks_if, "odd_stacks": odd_stacks_if, "nimsum": nimsum_if}

## Action

In [14]:

def all_get(state: Nim, row: int) -> Nimply:
    num_objects = state.rows[row]
    return Nimply(row, num_objects)

def one_get(state: Nim, row: int) -> Nimply:
    num_objects = 1
    return Nimply(row, num_objects)

def half_get(state: Nim, row: int) -> Nimply:
    num_objects = state.rows[row]/2
    return Nimply(row, num_objects)

def rand_get(state: Nim, row: int) -> Nimply:
    num_objects = random.randint(1, state.rows[row])
    return Nimply(row, num_objects)

def nimsum_get(state: Nim, row: int) -> Nimply:
    rows=game.rows
    totNimSum=nimSum(rows)
    if(totNimSum!=0):
        nonOneRow=checkMisere(rows)
        if nonOneRow!=-1:
            if len([_ for _ in rows if _!=0])%2==0:
                return Nimply(nonOneRow, rows[nonOneRow])
            else:
                return Nimply(nonOneRow, rows[nonOneRow]-1)
        for i,row in enumerate(rows):
            lineNimSum=nimSum([totNimSum,row])
            if(lineNimSum<row):
                return Nimply(i,row-lineNimSum)
    return rand_get(state, row)

actions = {"all": all_get, "one": one_get, "half": half_get, "rand": rand_get, "nimsum": nimsum_get}

## Place

In [15]:
def rand_place(state: Nim) -> int:
    row = random.choice([r for r, c in enumerate(state.rows) if c > 0])
    return row

def most_place(state: Nim) -> int:
    row = state.rows.index(max(state.rows))
    return row

def least_place(state: Nim) -> int:
    row = state.rows.index(min(state.rows))
    return row

def nimsum_place(state: Nim) -> int:
    rows=game.rows
    totNimSum=nimSum(rows)
    if(totNimSum!=0):
        nonOneRow=checkMisere(rows)
        if nonOneRow!=-1:
            return nonOneRow
        for i,row in enumerate(rows):
            lineNimSum=nimSum([totNimSum,row])
            if(lineNimSum<row):
                return i
    return rand_place(state)

places = {"rand": rand_place, "most": most_place, "least": least_place, "nimsum": nimsum_place}

## Fitness

In [16]:
def evaluate(genome: list(Gene)) -> float:
    win_count = 0
    for _ in range(eval_amount):
        while not game.endTest():
            turn = 1 - turn
            if not turn:
                game.nimming(play(genome))
            else:
                game.nimming(pure_random(game))
        if turn:
            win_count += 1
    return win_count/eval_amount

## Full Strategies

In [17]:
def pure_random(state: Nim) -> Nimply:
    row = random.choice([r for r, c in enumerate(state.rows) if c > 0])
    num_objects = random.randint(1, state.rows[row])
    return Nimply(row, num_objects)

def dumb(state: Nim) -> Nimply:
    possible_moves = [(r, o) for r, c in enumerate(state.rows) for o in range(1, c + 1)]
    return Nimply(*max(possible_moves, key=lambda m: (-m[0], m[1])))

def expert(game) -> Nimply:
    rows=game.rows
    totNimSum=nimSum(rows)
    if(totNimSum!=0):
        nonOneRow=checkMisere(rows)
        if nonOneRow!=-1:
            if len([_ for _ in rows if _!=0])%2==0:
                return Nimply(nonOneRow,rows[nonOneRow])
            else:
                return Nimply(nonOneRow, rows[nonOneRow]-1)
        for i,row in enumerate(rows):
            lineNimSum=nimSum([totNimSum,row])
            if(lineNimSum<row):
                print(f"removing {row-lineNimSum} elements from row {i}")
                return Nimply(i,row-lineNimSum)
    return pure_random(game)

def human_play(game) -> Nimply:
    while 1:
        str=input("Enter row index and number of elements to remove: ")
        rowIdx, n=str.split(" ")
        if int(rowIdx)>=len(game.rows):
            print("Invalid row!")
        elif int(n)>game.rows[int(rowIdx)]:
            print("Invalid number, convert to maximum!")
            n=game.rows[int(rowIdx)]
            return Nimply(int(rowIdx), int(n))
        else:
            return Nimply(int(rowIdx),int(n))
        
def lose_game() -> Nimply:
    return None

def play(state: Nim, genome: list(Gene)) -> Nimply:
    for g in genome:
        if (conditions[g.condition](state)):
            return Nimply(actions[g.action](state, places[g.place](state)))
    return lose_game() 

## GA Functions

In [ ]:
def generate_population(population_size):
    population = list()
    for _ in range(population_size):
        genome = list()
        for _ in range(genome_size):
            gene = (random.choice(conditions.keys()), random.choice(actions.keys()), random.choice(places.keys()))
            genome.push(gene)
        population.push(Individual(genome, evaluate(genome)))
    return population

def mutation(indiv: Individual) -> Individual:
    mutable = [conditions, actions, places]
    
    mut_gene_idx = random.randrange(0, genome_size)
    mut_attr_idx = random.randrange(0, len(Gene))
    attr = indiv.genome[mut_gene_idx][mut_attr_idx]
    
    acceptable = mutable[mut_attr_idx].keys().remove(attr)
    new_gene = random.choice(acceptable)
    new_genome = indiv.genome[:mut_gene_idx] + new_gene + indiv.genome[mut_gene_idx+1:]
    return Individual(new_genome, evaluate(new_genome))
        
def cross_over(i1: Individual, i2: Individual) -> Individual:
    #TODO
    pass
    
def evolution():
    #TODO
    pass

In [ ]:
if __name__ == '__main__':
    game=Nim(5)
    game.board()
    print("current nim sum: ",nimSum(game._rows))
    turn=1
    while not game.endTest():
        turn = 1 - turn
        if not turn:
            print("Your turn:")
            game.nimming(human_play(game))
        else:
            print("Agent turn:")
            game.nimming(expert(game))
        game.board()
    if not turn:
        print("You lost")
    else:
        print("Agent lost")